In [1]:
import pandas as pd
from transformers import BertTokenizerFast
import conllu
from collections import Counter
from tqdm.notebook import tqdm
import json
from itertools import chain

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
tqdm.pandas()

In [3]:
tok = BertTokenizerFast.from_pretrained('small-bert-tokenizer')

In [4]:
# res = []
# with open('all_verbs.jsonl', 'w', encoding='utf-8') as new:
#     with open('Biblioteka_prikluchenij.txt.conllu', "r", encoding="utf-8") as f:
#         for sent in tqdm(conllu.parse_incr(f)):
#             for token in sent:
#                 if token['upos'] == 'VERB':
#                     new.write(json.dumps(dict(token), ensure_ascii=False) + '\n')
#                     res.append(dict(token))

# df = pd.DataFrame(res)
# df = df.drop(columns=['id', 'head', 'deps', 'misc', 'xpos'])
# fts = pd.json_normalize(df.feats)
# new = df.join(fts)
# new.to_csv('all_verbs.csv', index=False)

In [5]:
new = pd.read_csv('all_verbs.csv')
new.form = new.form.apply(str.lower)
forms = new[new.VerbForm == 'Fin'][['form']].drop_duplicates()
forms['tokens'] = forms.form.progress_apply(tok.tokenize)
new = new.merge(forms, on='form')

  0%|          | 0/73647 [00:00<?, ?it/s]

In [6]:
new.head(1)

,form,lemma,upos,feats,deprel,Aspect,Mood,Number,Person,Tense,VerbForm,Voice,Case,Gender,Variant,Animacy,tokens
0,рассказывает,рассказывать,VERB,"{'Aspect': 'Imp', 'Mood': 'Ind', 'Number': 'Si...",root,Imp,Ind,Sing,3.0,Pres,Fin,Act,NaN,NaN,NaN,NaN,"[рассказ, ##ывает]"


In [7]:
new = new.drop_duplicates(subset=['form', 'lemma', 'upos', 'deprel', 'Aspect', 'Mood', 'Number',
       'Person', 'Tense', 'VerbForm', 'Voice', 'Case', 'Gender', 'Variant', 'Animacy'])
new = new[['form', 'lemma', 'Aspect', 'Mood', 'Number', 'Person', 'Tense', 'Voice', 'tokens']]
new = new.drop_duplicates(['form', 'lemma', 'Aspect', 'Mood', 'Number', 'Person', 'Tense', 'Voice'])
new['last'] = new.tokens.apply(lambda x: x[-1])

In [8]:
# st = new.groupby([ 'Number', 'Person'], as_index=False).value_counts(['last'])

In [9]:
parad = {
    'Plur,1': ['ем', 'им'],
    'Plur,2': ['ите', 'ете'],
    'Plur,3': ['ут', 'ют', 'ат', 'ят'],
    'Sing,1': ['у', 'ю'],
    'Sing,2': ['ешь', 'ишь'],
    'Sing,3': ['ет', 'ит'],
}
ends = ['##' + el for el in chain.from_iterable(parad)]

In [10]:
for_exp = new[(new['last'].isin(ends)) | (new.form.str.contains('(ишь|ешь)$'))].reset_index(drop=True)

/var/folders/d0/fv07hvxd1nb4x_xxdnrl4d2h0000gq/T/ipykernel_59622/3734836427.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  for_exp = new[(new['last'].isin(ends)) | (new.form.str.contains('(ишь|ешь)$'))].reset_index(drop=True)


In [11]:
cur = new[['form', 'tokens', 'last']].reset_index(drop=True)
cur.head(1)

,form,tokens,last
0,рассказывает,"[рассказ, ##ывает]",##ывает


In [14]:
def get_categ(word):
    res = {}
    for cat in parad:
        res[cat] = [word[0] + el for el in parad[cat]]
    return res

In [17]:
prt = cur.progress_apply(get_categ, axis=1, result_type='expand')
cur = cur.join(prt)

  0%|          | 0/79393 [00:00<?, ?it/s]

In [20]:
for col in cur.columns[3:]:
    cur = cur.explode(col)
    print(cur.shape)

(158786, 9)
(317572, 9)
(1270288, 9)
(2540576, 9)
(5081152, 9)
(10162304, 9)


In [63]:
def tok_all(line):
    res = [tok.tokenize(el) for el in line[3:]]
    return res

In [66]:
cur['new_tokens'] = cur.progress_apply(tok_all, axis=1)

  0%|          | 0/10162304 [00:00<?, ?it/s]

In [67]:
# cur.to_csv('result_df.csv', index=False)

In [7]:
cur = pd.read_csv('result_df.csv')
cur.head()

,form,tokens,last,"Plur,1","Plur,2","Plur,3","Sing,1","Sing,2","Sing,3",new_tokens,same_count,suffixes
0,рассказывает,"['рассказ', '##ывает']",##ывает,рассказываетем,рассказываетите,рассказываетут,рассказываету,рассказываетешь,рассказываетет,"[['рассказ', '##ывает', '##ем'], ['рассказ', '...",6,"('ем', 'ите', 'ут', 'у', 'ешь', 'ет')"
1,рассказывает,"['рассказ', '##ывает']",##ывает,рассказываетем,рассказываетите,рассказываетут,рассказываету,рассказываетешь,рассказываетит,"[['рассказ', '##ывает', '##ем'], ['рассказ', '...",6,"('ем', 'ите', 'ут', 'у', 'ешь', 'ит')"
2,рассказывает,"['рассказ', '##ывает']",##ывает,рассказываетем,рассказываетите,рассказываетут,рассказываету,рассказываетишь,рассказываетет,"[['рассказ', '##ывает', '##ем'], ['рассказ', '...",6,"('ем', 'ите', 'ут', 'у', 'ишь', 'ет')"
3,рассказывает,"['рассказ', '##ывает']",##ывает,рассказываетем,рассказываетите,рассказываетут,рассказываету,рассказываетишь,рассказываетит,"[['рассказ', '##ывает', '##ем'], ['рассказ', '...",6,"('ем', 'ите', 'ут', 'у', 'ишь', 'ит')"
4,рассказывает,"['рассказ', '##ывает']",##ывает,рассказываетем,рассказываетите,рассказываетут,рассказываетю,рассказываетешь,рассказываетет,"[['рассказ', '##ывает', '##ем'], ['рассказ', '...",6,"('ем', 'ите', 'ут', 'ю', 'ешь', 'ет')"


In [8]:
cur.form.unique().shape

(73647,)

In [69]:
cur.shape

(10162304, 10)

In [70]:
def check_line(line):
    count = 0
    for el in line[9]:
        if line[1] == el[:len(line[1])]:
            count += 1
    return count

In [73]:
cur['same_count'] = cur.progress_apply(check_line, axis=1)

  0%|          | 0/10162304 [00:00<?, ?it/s]

In [77]:
def get_suffixes(line):
    res = []
    for el in line[3:9]:
        res.append(el[len(line[0]):])
    return tuple(res)

In [79]:
cur['suffixes'] = cur.progress_apply(get_suffixes, axis=1)

  0%|          | 0/10162304 [00:00<?, ?it/s]

In [85]:
cur.to_csv('result_df.csv', index=False)

In [87]:
cur = cur[['same_count', 'suffixes']]

In [88]:
cur.head()

,same_count,suffixes
0,6,"(ем, ите, ут, у, ешь, ет)"
0,6,"(ем, ите, ут, у, ешь, ит)"
0,6,"(ем, ите, ут, у, ишь, ет)"
0,6,"(ем, ите, ут, у, ишь, ит)"
0,6,"(ем, ите, ут, ю, ешь, ет)"


In [89]:
st = cur.groupby('suffixes').value_counts(['same_count'])
st.head()

suffixes                   same_count
(ем, ете, ат, у, ешь, ет)  6             53503
                           0              9524
                           1              6328
                           2              5600
                           5              1575
dtype: int64

In [95]:
st = pd.DataFrame(st).reset_index()

In [102]:
st[st.same_count == 6].sort_values(0, ascending=False).style

,suffixes,same_count,0
139,"('ем', 'ете', 'ют', 'ю', 'ешь', 'ет')",6,56074
56,"('ем', 'ете', 'ут', 'у', 'ешь', 'ет')",6,54962
111,"('ем', 'ете', 'ют', 'у', 'ешь', 'ет')",6,54817
83,"('ем', 'ете', 'ут', 'ю', 'ешь', 'ет')",6,54815
825,"('им', 'ите', 'ют', 'ю', 'ишь', 'ит')",6,54448
0,"('ем', 'ете', 'ат', 'у', 'ешь', 'ет')",6,53503
28,"('ем', 'ете', 'ат', 'ю', 'ешь', 'ет')",6,53467
811,"('им', 'ите', 'ют', 'ю', 'ешь', 'ит')",6,52332
382,"('ем', 'ите', 'ют', 'ю', 'ишь', 'ит')",6,52332
369,"('ем', 'ите', 'ют', 'ю', 'ешь', 'ит')",6,52330


In [109]:
st = pd.pivot_table(st, values=0, index=['suffixes'],
                       columns=['same_count']).fillna(0).sort_values([6, 5, 4, 3, 2, 1, 0], ascending=False).reset_index()

In [110]:
# st.to_csv('final_stats.csv', index=False)

In [ ]:
('ем', 'ете', 'ют', 'ю', 'ешь', 'ет')